In [ ]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))
import openvino


import videoHtml
videoHtml.videoHTML('Manufacturing', ['original_videos/Manufacturing.mp4'])


MODEL_PATH = '/data/models/intel/person-detection-retail-0013/FP16/person-detection-retail-0013'
VIDEO = '/data/resources/manufacturing.mp4'
QUEUE = '/data/queue_param/manufacturing.npy'
PEOPLE = 5


def submit(device, node):
    device_postfix = device
    if device.startswith('HETERO:FPGA'):
        device_postfix = 'FPGA'
        
    job_name = 'JOB_MANUFACTURING_' + device_postfix
    output = '/output/results/manufacturing/' + device_postfix
    params = '{} {} {} {} {} {}'.format(MODEL_PATH, device, VIDEO, QUEUE, output, PEOPLE)

    job_id = !qsub queue_job.sh -l nodes=1:{node} -d . -F "{params}" -N {job_name}
    return job_id


# Submit job to the queue
cpu_job_id = submit('CPU', 'tank-870:i5-6500te')
print(cpu_job_id[0])


import liveQStat
liveQStat.liveQStat()
import get_results
get_results.getResults(cpu_job_id[0], filename='output.tgz', blocking=True)


!tar xzf output.tgz
!cat stdout.log
!cat stderr.log